# Autonomous Vehicle Learning Curve Estimate

This notebook estimates the learning curve for unstructured terrain autonomous vehicles.  The analysis uses the incident rate for on-road autononmous vehicles to estimate the incident rate for off-road AVs.

The on-road AV data is made public by the California DMV.

In [3]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import Image
from IPython.core.display import display, HTML

## The Learning Curve

We can estimate the learning curve (i.e. the number of incidents per hour driven in the field) from the DMV data.  The following chart summarizes the miles driven by AV manufacturer in 2020 along with the miles driven between incidents.

In [2]:
url ='https://cdn.statcdn.com/Infographic/images/normal/17144.jpeg'
Image(url= url)

NameError: name 'Image' is not defined

### Data Cleansing

We can remove the highest performing AV makers from the mix. The Chinese AV makers in particular are probably gaining experience outside of California and this presents unrealistic intervention rates.

In [ ]:
# worst case incidents / mile driven
miles = [#(628839, 29945),
         (770049, 28520),
         (225496, 10738),
        (55370,5034),
         #(40734,20367),
         #(21037,10519),
         (13014,6507),
         (10401,5201),
        ]
miles = sorted(miles, key=lambda k:k[0])
df = pd.DataFrame(miles, columns=['miles','miles_between_interventions'])

# Assumption

We can assume an average speed of 15 mph to translate the miles driven into hours of operation (a more relevant metric for off-road vehicles given the reduced speed).  From the hours of operation, we can compute the mean time between failures (MTBF):


In [ ]:
df['hours'] = df['miles'] / 15
df['mtbf'] = df['miles_between_interventions'] / 15
df

In [ ]:
fig = px.scatter(df, 
                 x='hours',
                 y='mtbf',
                 trendline="ols",
                 title='Mean Time Between Failures (Human Interventions)<br>vs Hours Driven in the Field'
                )
fig.show()
results = px.get_trendline_results(fig)
#results.iloc[0].values[0].summary()

## Learning Curve Equation

From the above data, we get the following learning curve equation:

$$
MTBF = .03 \times h + 307
$$

where:  
$MTBF$ Mean time between failure (or human interventions)  
$h$ is the cumulative number hours of experience

In [ ]:
reg = stats.linregress(x=df['hours'],y=df['mtbf'])
reg

## Learning curve

Let's assumes the following number of tractors will ship over time:

Year | count
--|--
2022 | 1
2023 | 33
2024 | 161
2025 | 585
2026 | 1585
2027 | 6000
2028 | 15000

The following is an estimate of the cost per tractor unit:

In [ ]:
tractor_count = [2, 33, 161, 585, 1585, 6000,15000]
tractor_cost = [35000,29000,23000,20000,17500,12500,9500]

### Assumptions

Let's assume an autonomous tractor in production has the following operating profile:

- hours per day: 8
- days per week: 7
- unit forecast (see above)
- the opex is ~ 300 USD per month or 3,600 USD / year

In [ ]:
cum_count = 0
cum_hours = 0
learnings = 0
rpt = []
for i,n in enumerate(tractor_count):
    cost = tractor_cost[i]
    cum_count += int((n / 2))
    hours = cum_count * 8 * 52 * 7
    cum_hours += hours
    
    mtbf = reg.slope * cum_hours + reg.intercept
    learnings += hours / mtbf
    opex = 3600
    r= {'year':i + 2022,
        'count':n,
        'cum_count':cum_count,
        'hours': hours,
        'cum_hours': cum_hours,
        'mtbf': mtbf,
        'learnings': int(learnings),
        'cost_per_tractor':cost,
        'capex': cost * n,
        'opex': opex,
}
    rpt += [r]
a = pd.DataFrame(rpt)
a

In [ ]:
# Cost of Operation

In [ ]:
a['investment'] = a['capex'] + a['opex']
a['cum_investment'] = a['investment'].cumsum()
a['cost_per_learning'] = a['cum_investment'] / a['learnings']
a

## Conclusions

As Arcadia ships more units in the field it gains operating hours and over time it runs into field issues (i.e. learnings) at a rate prescribed by the learning curve.  As the frequency of incidents decreases, each learning becomes more valuable:

In [ ]:
fig = px.bar(a, 
             x='year', 
             y='cost_per_learning',
             title='Avg Cost per New Learning over Time').show()

In [ ]:
# cost to catch up

### Cost to Catch Up

As Arcadia gains experience in the field, it becomes more and more difficult for other firms to catch up.  If we estimate that a competitor trying to pivot would take 18 to 24 months to effect the pivot, the cost of catching up (excluding the R&D investment) is shown below. This cost includes the CapEx to build tractors and the Opex to run tractors in the field.  This number also assumes competitors have the same learning curve as Arcadia.

In [ ]:
a['cost_to_catch_up'] = a['cum_investment'].shift(-2)
a['best_case_competitor_hours'] = a['cum_hours'].shift(2)
fig = px.bar(a.head(5), 
             x='year', 
             y='cost_to_catch_up',
             title='Field Investment Required (USD) to Catch Up').show()

### Data Advantage

In 2023, Arcadia will have 300k hours of field experience before competitors are even out of the gate.  To catch-up, competitors would have to:

1. pursuade partners willing to spend time running a new solution (a difficult task when a working solution is already available), and
2. invest $50M in field operations

The competitive gap would go up non-linearly -- i.e. every year the number of hours required goes up exponentially.

In [ ]:
fig = go.Figure()
# Create and style traces
fig.add_trace(go.Scatter(x=a['year'], y=a['cum_hours'], name='Arcadia',
                         line=dict(color='royalblue', width=4)))

fig.add_trace(go.Scatter(x=a['year'], y=a['best_case_competitor_hours'], name='Others',
                         line=dict(color='firebrick', width=4)))

# Edit the layout
fig.update_layout(title='Unstructured Terrain<br>Operating Hours',
                   xaxis_title='Year',
                   yaxis_title='Field Operating Hours')


fig.show()